#**Geospatial analyses using WaPOR data**
#Topic 3 - Notebook 4: Average Seasonal Evapotranspiration for area of interest
In this notebook we will use the WaPOR AETI data downloaded through the Topic3_N01 notebook and shapefile of wheat field in Gezira irrigation scheme and calculate the seasonal AETI per plot and decadal timeseries of AETI for each plot.   
The steps are:
1. Install and load necessary packages to manage raster files
2. Load the .zip file from Topc3_N01 to this session (or create a new .zip)
3. Unzip files
4. Get the list of files from uploaded zipped file
5. Perform the temporal aggregation (sum) of the decadal AETI
6. Zonal statistics (compute average AETI per plot for one filed)
7. Saving the dataframes computed above to csv files and downlaod them to local folder
8. Exercises
9. Bonus - Compute time series of decadal AETI per plot.





---



### **Step 1**

In [ ]:
# Install required packages
%%capture
!pip install --upgrade xarray  --quiet
!pip install --upgrade geopandas --quiet
!pip install --upgrade rioxarray --quiet

In [ ]:
# Import the required packages
from osgeo import gdal
import xarray as xr
import rioxarray as rio
import numpy as np
import pandas as pd
import geopandas as gpd
import glob
import os

### **N.B.**
>In this notebook we used additional python packages called <font color='steelblue'>**`Xarray`**</font> and <font color='steelblue'>**`rioxarray`**</font>. <font color='steelblue'>**`Xarray`**</font> is a python library which simplifies working with labelled multi-dimension arrays such as stack of rasters and<font color='steelblue'>**`rioxarray`**</font> is an Xarray extension that allows reading and writing a wide variety of geospatial image formats compatible with Geographic Information Systems (GIS). Introducing these packages is out of the scope of this notebook, but **[here](https://tutorial.xarray.dev/overview/xarray-in-45-min.html)** and **[here](https://corteva.github.io/rioxarray/html/getting_started/getting_started.html)** you can find good introductions about the packages which provid you the basic understanding.



---





---



### **Step 2 - Upload .zip to your colab files**
We will now upload the .zip file.
Execute the command below, click the `Choose Files` button, navigate to where you have saved the .zip file in your local drive and select the zipped file.

In [ ]:
#To upload file.
from google.colab import files
uploaded = files.upload()



---



### **Step 3 - Unzip files**
Unzip the file you have uploaded.
The command is:
 !unzip 'file_path.zip' -d 'folder_to_unzip_path'
Replace the file name in the cell below with the name of your own file. You can find the path to your file by clicking on the 3 dots next to the file name in your file explorer to the left and selecting *Copy path*. (If you do not see your files to the left, click on the folder icon to expand the Files panel)

In [ ]:
# unzip the uploaded zipfile
!unzip '/content/WaPOR_data.zip' -d '/content/'


### **Step 4 - Get the list of files from uploaded zipped file**
The list of files can be obtained in the following way.
```python
# Example to get all the files with '.tif' extension
glob.glob('/content/WaPOR_data/L3-AETI-D/*.tif')
```



In [ ]:
rst_fh = glob.glob('/content/WaPOR_data/L3-AETI-D/*.tif')
rst_fh # this holds the list of raster files found in the 'L3-AETI-D.zip' file.



---



### **Step 5: Perform the temporal aggregation (sum) of the decadal AETI**

We create an xarray dataarray from the raster files so that any further computation is easier to do

In [ ]:
# Create an xarray dataarray from the raster files so that any further computation is easier to do

# get the time of the rasters from filenames
time = [pd.to_datetime(i[-14:-4], format='%Y-%m-%d') for i in rst_fh]

# read the first raster to get the crs and scale_factor
tif_0 = rio.open_rasterio(rst_fh[0])
crs = tif_0.rio.crs
scale_factor = tif_0.attrs['scale_factor']

# read each raster and concatenate them to create an xarray dataarray
ds = xr.concat([rio.open_rasterio(f) for f in rst_fh], dim=time)
ds = ds.squeeze(dim = 'band', drop = True)
ds = ds.drop_vars(['spatial_ref'])

# rename the dimnesions of the dataarray
ds = ds.rename({'concat_dim':'time','x': 'longitude','y': 'latitude'})

# multiply the data by the scale factor
ds = ds.where(ds !=ds.attrs['_FillValue'])*scale_factor

# write CRS information to the dataarray
ds = ds.rio.write_crs(crs, inplace=True)

# update the attributes of the dataarray
attrs = ds.attrs
attrs.update({'start_date': '2022-10-01',
              'end_date': '2023-04-30',
              '_FillValue': np.nan})
attrs_to_delet = [i for i in attrs if 'STATISTICS_' in i]
attrs_to_delet.append('number_of_days')
attrs_new = {key: attrs[key] for key in attrs if key not in attrs_to_delet}

ds.attrs  = attrs_new

### Compute the seasonal sum of AETI

In [ ]:
# We can sum up all the decadal taster to seasonal raster.
seasonal_year='YE-{0}'.format('Nov')  # the season we use is from November 2022 to April 2023
# sum the decadal AETI to seasonal AETI
ds_season = ds.sum(dim='time',skipna=False)
ds_season = ds_season.drop_vars(['spatial_ref'])
ds_season = ds_season.rename('Seasonal AETI')
ds_season = ds_season.rio.write_crs(crs)
ds_season

### Update attributes of the seasonal dataarray



In [ ]:
attrs = ds_season.attrs
attrs.update({'start_date': '2022-10-01',
              'end_date': '2023-04-30',
              'units' : 'mm/sdeceason',
              'temporal_resolution' : 'Seasonal'})
attrs_to_delet = [i for i in attrs if 'STATISTICS_' in i]
attrs_to_delet.append('number_of_days')
attrs_new = {key: attrs[key] for key in attrs if key not in attrs_to_delet}

ds_season.attrs  = attrs_new

In [ ]:
# plot the season sum AETI
ds_season.plot()



---



## **Step 6 - Zonal statistics (compute average AETI per plot for one field)**

Upload and read area of interest defined by geojson file using geopandas

In [ ]:
#To upload file.
from google.colab import files
uploaded = files.upload()

In [ ]:
## Read polygons of fields in the are of interest for wheat fields
import warnings; warnings.filterwarnings('ignore', 'GeoSeries.notna', UserWarning)

aoi = r"/content/Fields.geojson" ## give the correct path of the geojson file here
gdf = gpd.read_file(aoi)
gdf

In [ ]:
# Plotting the plots
gdf.plot()

As you can see from the above geopandas dataframe, there are 8 fields with several plot per filed.

You can get the name of the fields using the following code.

In [ ]:
# get the name of the fields
np.unique(gdf['layer'])

We will use the field **`'FakiMusa6'`** as an example.we can select the rows beloging to this field.

In [ ]:
## Select one of the fields (FakiMusa6)
farm_field = gdf[gdf['layer']=='FakiMusa6']
farm_field


This field has several plots. you gen the number of the plots suing the code in the following cell.

In [ ]:
# Number of Polygons in FakiMusa6 field
len(farm_field)

In [ ]:
# Iterate through the plots and use the plots polygons to clip the AETI dataarray

# creat empty lists to hold the values for each plot
AETI_per_plot = [] # to hold the AETI per plot
plot_id = [] # to hold ID of the plots
plot_area = [] # to hold the area of plots

for index, row in farm_field.iterrows(): # iteration through the farm_field dataframe rows
  shdf = farm_field.loc[[index]]

  if(not shdf.geometry.item().is_empty): # check if the polygon of the plot is not empty
      da = ds_season.rio.clip(shdf.geometry, farm_field.crs, all_touched=True)
      # compute AAETI of the polygon
      da = da.mean(dim=['latitude','longitude']).data

      AETI_per_plot.append(da) # add the AETI per polt to the AETI_per_plot list
      plot_id.append(shdf.id.values[0]) # add the ID of a plot to plot_id list
      plot_area.append(shdf.area.iloc[0]) # add area of the plot polygon to the plot_area list

# # round the numbers in AETI and area for each polygon
AETI_per_plot = [ np.around(i,2) for i in AETI_per_plot]
plot_area = [np.around(i,2) for i in plot_area]

A dataframe can be ceated using the thre lists we created in the above cell as follows:

In [ ]:
# creat a Pandas dataframe from the AETI, area and ID of each polygon
data={'Plot_ID':plot_id,'Plot_mean_AETI [mm/season]':AETI_per_plot,
        'Plot_area [meter square]':plot_area}
df = pd.DataFrame(data)
df = df.sort_values('Plot_ID',ascending=True)
df


To calculate the min, max and mean of the Plot_mean_AETI and Plot_area columns we use the following Panda functions:
```python
    df.iloc[:,1:] # Selects the the colusm of the dataframe starting from the secon column
    .agg(['min', 'max', 'mean']) # aggregate function to calculate the min, max and mean of the selected dataframe columns
    .round() # function to round the values of the colums
    .astype(int) # to chnage the data type of the columns to integre

```

In [ ]:
# applying the above functions to get minimum, maximum and mean of the AETI and the area
mm = df.iloc[:,1:].agg(['min', 'max', 'mean']).round().astype(int)
mm



---



## 7 -  Saving the dataframes computed above to csv files and downlaod them to local folder:

In [ ]:
# for dataframe df
farm_name = farm_field.layer.iloc[0]
file_name1 = f'AETI_and_Area_per_plot_for_{farm_name}_field.csv'
df.to_csv(file_name1)

# for dataframe mm
file_name2 = f'Stat_for_{farm_name}_field.csv'
mm.to_csv(file_name2)

#download the two csv files
files.download(file_name1)
files.download(file_name2)



---



<div class="alert alert-success">

## **8 - EXERCISE**:

>Do the zonal statistics for field **`'Eltukl10'`** and get the minimum, maximum and mean of the average AETI and the area of plots.

>Write down the values (rounded to the integre values) on a peice paper, you will need them to answer questions in the OCW.


<details>
  <summary>Hints</summary>

>Select the field **`'Eltukl10'`** from the geodataframe (at the 13th cell) and run the last three cells.

</details>
    
</div>





---



## **9 - Bonus - decadal AETI time series**
In you want to get a timeseries of decadal AETI per plot, you can run the following three cells and asa exercise try to do the same for field **`'Eltukl10'`**

### **Steps:**


1. Use the same filed as above (FakiMusa6) and the decadal data
2. Iterate over the plot of the field, clip the dataarray by each plot and compute the average AETI per plot
3. Save the result of each iteration in a dictionary (decadal_AETI)
4. Convert the dictionary to a dataframe
5. assign the dataarray time to index of the dataframe
6. Plot the dataframe



In [ ]:
decadal_AETI = {}

ds = ds.where(ds!=ds.attrs['_FillValue'])

for index, row in farm_field.iterrows():
  shdf = farm_field.loc[[index]]

  if(not shdf.geometry.item().is_empty):
      da = ds.rio.clip(shdf.geometry, farm_field.crs, all_touched=True, drop=True)
      # compute AAETI of the polygon
      d2 = da.mean(dim=['latitude','longitude'], skipna = True)
      decadal_AETI[shdf.id.values[0]] = d2.values


In [ ]:
df2 = pd.DataFrame(decadal_AETI)
df2.index = ds.time.values
df2.round()

In [ ]:
# Plot the dataframe
df2.plot(figsize = (12,6))

In [ ]:
# save the dataframe and download to local folder
file_name3 = f'decadal_AETI_timeseries_for_{farm_name}_field.csv'
df2.to_csv(file_name3)

#download the two csv files
files.download(file_name3)

In [ ]:
# # if you want to delete a folder use the code belwo.
# !rm -rf /content/L3-AETI-D
# !rm -rf /content/tifs
# !rm -rf /content/L3-AETI-D.zip